In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [4]:

from FRVRS import nu
import os.path as osp
import re
import os
import pandas as pd


# Analyze Util Function Usage Frequency

In [5]:

nu.delete_ipynb_checkpoint_folders()

In [3]:

# Create a regular expression of all the utils functions
defs_regex = re.compile(r'^    def ([a-z]+)([a-z_]+)\(\s*self,\s+([^\)]+)\):', re.MULTILINE)
functions_list = []
for file_path in ['frvrs_utils.py', 'notebook_utils.py']:
    with open(f'../py/{file_path}', 'r', encoding=nu.encoding_type) as f: text = f.read()
    for match_obj in defs_regex.finditer(text):
        functions_list.append(''.join([s[0] for s in file_path.split('.')[0].split('_')]) + '\\.' + match_obj.group(1) + match_obj.group(2))

functions_regex = re.compile('\\b(' + '|'.join(functions_list) + ')\\b')

FileNotFoundError: [Errno 2] No such file or directory: '../py/frvrs_utils.py'

In [ ]:

# Tabulate util function usage frequency
usage_frequency_dict = {}
for sub_directory, directories_list, files_list in os.walk(nu.github_folder):
    if '.ipynb_checkpoints' not in sub_directory:
        for file_name in files_list:
            if file_name.endswith('.ipynb') and not ('Attic' in file_name):
                file_path = osp.join(sub_directory, file_name)
                with open(file_path, 'r', encoding=nu.encoding_type) as f:
                    lines_list = f.readlines()
                    for line in lines_list:
                        match_obj = functions_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            usage_frequency_dict[fn] = usage_frequency_dict.get(fn, 0) + 1

In [ ]:

usage_frequency_df = pd.Series(usage_frequency_dict).to_frame().reset_index().rename(columns={0: 'usage_frequency', 'index': 'function_call'})
usage_frequency_df.sort_values('usage_frequency', ascending=False).head(20)

In [ ]:

mask_series = (usage_frequency_df.usage_frequency == 1)
usage_frequency_df[mask_series]